<a href="https://colab.research.google.com/github/MarkmeDEV/Google-Colab-Entry-and-Exit/blob/main/Entry_And_Exit_of_tenant_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:

tenant_entry_list_file = files.upload()
tenantEntryfile = list(tenant_entry_list_file.keys())[0]

tenant_exit_list_file = files.upload()
tenantExitFile = list(tenant_exit_list_file.keys())[0]


if tenantEntryfile.endswith('.xlsx'):
  entry_df = pd.read_excel(tenantEntryfile, engine='openpyxl')
elif tenantEntryfile.endswith('.csv'):
  entry_df = pd.read_csv(tenantEntryfile)
elif tenantEntryfile.endswith('.xls'):
  entry_df = pd.read_excel(tenantEntryfile, engine='xlrd')
else:
  raise ValueError('Unsupported file format')

if tenantExitFile.endswith('.xlsx'):
  exit_df = pd.read_excel(tenantExitFile, engine='openpyxl')
elif tenantExitFile.endswith('.csv'):
  exit_df = pd.read_csv(tenantExitFile)
elif tenantExitFile.endswith('.xls'):
  exit_df = pd.read_excel(tenantExitFile, engine='xlrd')
else:
  raise ValueError('Unsupported file format')

entry_df['Date'] = pd.to_datetime(entry_df['Date'], errors='coerce')
entry_df['Date'] = entry_df['Date'].fillna(method='ffill')
entry_cleaned = entry_df.dropna(subset=['Name'])
entry_cleaned.sort_values(by=['Name', 'Date', 'Time'], ascending=True, inplace=False)

exit_df['Date'] = pd.to_datetime(exit_df['Date'], errors='coerce')
exit_df['Date'] = exit_df['Date'].fillna(method='ffill')
exit_cleaned = exit_df.dropna(subset=['Name'])
exit_cleaned.sort_values(by=['Name', 'Date', 'Time'], ascending=True, inplace=False)

combined_df = pd.concat([entry_cleaned, exit_cleaned])
combined_df = combined_df.sort_values(by=['Date', 'Name'], ascending=[True, True])

df_filtered = combined_df[~((combined_df['Description'] == 'Denied Unauthorized Area') |
                          (combined_df['Description'] == 'Denied User Access Suspended') |
                          (combined_df['Description'] == 'Data Mismatch'))]

df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], errors='coerce')

df_filtered['Time'] = df_filtered['Time'].str.strip()

try:
    # Try parsing time in 24-hour format first
    df_filtered['Time'] = pd.to_datetime(df_filtered['Time'], format='%H:%M:%S', errors='raise').dt.time
except ValueError:
    # If the 24-hour format fails, try 12-hour format with AM/PM
    df_filtered['Time'] = pd.to_datetime(df_filtered['Time'], format='%I:%M:%S %p', errors='coerce').dt.time

df_filtered = df_filtered.sort_values(by=['Date', 'Time'], ascending=[True, True])

df_filtered.head(100)

cleaned_data = []

# Step 2: Group by 'Name' and 'Date' to process each person for each day
grouped = df_filtered.groupby(['Name', 'Date'])

for name_date, group in grouped:
    entries = group[group['Description'].str.contains('Entry')].sort_values(by='Time')
    exits = group[group['Description'].str.contains('Exit')].sort_values(by='Time')

    # Case 1: Multiple Entries, Single Exit
    if len(entries) > 1 and len(exits) == 1:
        # Keep the first entry and the exit
        cleaned_data.append(entries.iloc[0])  # First entry
        cleaned_data.append(exits.iloc[0])    # Exit

    # Case 2: Multiple Entries, No Exit
    elif len(entries) > 1 and len(exits) == 0:
        # Keep only the first entry, remove the rest
        cleaned_data.append(entries.iloc[0])  # First entry

    # Case 3: Multiple Exits, No Entry
    elif len(exits) > 1 and len(entries) == 0:
        # Keep only the first exit, remove the rest
        cleaned_data.append(exits.iloc[0])  # First exit

    # Case 4: Mixed Entries and Exits
    else:
        # For alternating entry-exit pairs, keep first entry for each exit
        if len(entries) > 0 and len(exits) > 0:
            cleaned_data.append(entries.iloc[0])  # First entry
            cleaned_data.append(exits.iloc[0])    # First exit

            # Continue processing the rest
            if len(entries) > 1:
                for i in range(1, min(len(entries), len(exits))):
                    cleaned_data.append(entries.iloc[i])
                    cleaned_data.append(exits.iloc[i])

cleaned_df = pd.DataFrame(cleaned_data)

cleaned_df.reset_index(drop=True, inplace=True)

# cleaned_df['DateTime'] = pd.to_datetime(cleaned_df['Date'].astype(str) + ' ' + cleaned_df['Time'].astype(str))
cleaned_df.sort_values(by=['Date', 'Name'], ascending=[True, True], inplace=True)

cleaned_df.reset_index(drop=True, inplace=True)
# cleaned_df.drop(columns='DateTime', inplace=True)

# cleaned_df.head(100)
output_file = f"/content/cleaned_merge_two.csv"
cleaned_df.to_csv(output_file, index=False)

files.download(output_file)

# output_name = 'Exit Cleaned'
# output_file = f"/content/{output_name}.csv"
# df_cleaned.to_csv(output_file, index=False)

# files.download(output_file)

#  IF(ISBLANK(A2), " ", TEXT(MID(A2, SEARCH(",", A2) +2, LEN(A2)), "mm/dd/yyyy"))
# =IF(AND(C2="Granted Entry", OR(COUNTIFS(E$2:E2, E2, C$2:C2, "Granted Entry", F$2:F2, F2)=1, COUNTIFS(E$2:E2, E2, C$2:C2, "Granted Exit", F$2:F2, F2)>0)), "Entry", IF(C2="Granted Exit", "Exit", ""))
# =IF(G2="Exit", LOOKUP(2, 1/(($G$2:G2="Entry")*($F$2:F2=F2)*(ROW($A$2:A2)<ROW(A2))), $A$2:A2), "")
# =IF(G2="Entry", IF(COUNTIFS(F:F, F2, G:G, "Exit", C:C, "Granted Exit", D:D, D2)=0, "No Exit", ""), IF(G2="Exit", IF(COUNTIFS(F:F, F2, G:G, "Entry", C:C, "Granted Entry", D:D, D2)=0, "No Entry", ""), ""))

<ipython-input-48-f10274dcb034>:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  entry_df['Date'] = entry_df['Date'].fillna(method='ffill')
<ipython-input-48-f10274dcb034>:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  exit_df['Date'] = exit_df['Date'].fillna(method='ffill')
<ipython-input-48-f10274dcb034>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], errors='coerce')
<ipython-input-48-f10274dcb034>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
entry = files.upload()
exit = files.upload()

entry_filename = list(entry.keys())[0]
exit_filename = list(exit.keys())[0]

if entry_filename.endswith('.xlsx'):
  df_entry = pd.read_excel(entry_filename, engine='openpyxl')
elif entry_filename.endswith('.xls'):
  df_entry = pd.read_excel(entry_filename, engine='xlrd')
elif entry_filename.endswith('.csv'):
  df_entry = pd.read_csv(entry_filename)
else:
  raise ValueError('Unsupported file format')

if exit_filename.endswith('.xlsx'):
  df_exit = pd.read_excel(exit_filename, engine='openpyxl')
elif exit_filename.endswith('.xls'):
  df_exit = pd.read_excel(exit_filename, engine='xlrd')
elif exit_filename.endswith('.csv'):
  df_exit = pd.read_csv(exit_filename)
else:
  raise ValueError('Unsupported file format')

combined_df = pd.concat([df_entry, df_exit])
combined_df = combined_df.sort_values(by=['Date', 'Name'], ascending=[True, True])

df_filtered = combined_df[~((combined_df['Description'] == 'Denied Unauthorized Area') |
                          (combined_df['Description'] == 'Denied User Access Suspended') |
                          (combined_df['Description'] == 'Data Mismatch'))
                          # (combined_df['Name'] == 'Loc&Stor Office Use') |
                          # (combined_df['Name'] == 'Maint/Clean Edwin') |
                          # (combined_df['Name'] == 'Kaye Pasion') |
                          # (combined_df['Name'] == 'Samuel Peterson') |
                          # (combined_df['Name'] == 'Maint/Clean Manuel') |
                          # (combined_df['Name'] == 'Rommel Mendoza/Mai') |
                          # (combined_df['Name'] == 'Raymart Alcala') |
                          # (combined_df['Name'] == 'Maint/Clean Rennie') |
                          # (combined_df['Name'] == 'SG George Forte') |
                          # (combined_df['Name'] == 'Quinie Pulido') |
                          # (combined_df['Name'] == 'Jocelyn Figalan') |
                          # (combined_df['Name'] == 'Crystal Ruiz') |
                          # (combined_df['Name'] == 'Lyka PeÃƒÂ±aloza') |
                          # (combined_df['Name'] == 'Harvin Ramos') |
                          # (combined_df['Name'] == 'Lyka PeÃƒÂ±aloza'))
                          ]

df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], errors='coerce')

df_filtered['Time'] = df_filtered['Time'].str.strip()

try:
    # Try parsing time in 24-hour format first
    df_filtered['Time'] = pd.to_datetime(df_filtered['Time'], format='%H:%M:%S', errors='raise').dt.time
except ValueError:
    # If the 24-hour format fails, try 12-hour format with AM/PM
    df_filtered['Time'] = pd.to_datetime(df_filtered['Time'], format='%I:%M:%S %p', errors='coerce').dt.time

df_filtered = df_filtered.sort_values(by=['Date', 'Name', 'Time'], ascending=[True, True, True])


# grouped = df_filtered.groupby(['Name', 'Date'])

output_name = 'Entry and Exit Combined Sept 1 - 30'
output_file = f"/content/{output_name}.csv"
df_filtered.to_csv(output_file, index=False)

files.download(output_file)

Saving Entry Cleaned.csv to Entry Cleaned (1).csv


Saving Exit Cleaned.csv to Exit Cleaned (1).csv


<ipython-input-6-6b49c085478f>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], errors='coerce')
<ipython-input-6-6b49c085478f>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Time'] = df_filtered['Time'].str.strip()
<ipython-input-6-6b49c085478f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving Entry and Exit Combined Sept 1 - 30.csv to Entry and Exit Combined Sept 1 - 30.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>